# Code for the paper: 

## Efficient and Mobile Deep Learning Architectures for Fast Identification of BacterialStrains in Resource-Constrained Devices

### Architecture: EfficientNet B2
### Data: Original + Cross-Validation

#### Loading libraries

In [1]:
# Imports here
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import torch.nn.functional as F
import pandas as pd
import time
import numpy as np

from PIL import Image
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import KFold
from torch import nn
from torch import optim
from torch.utils.data import SubsetRandomSampler
from torchvision import datasets, transforms, models
from pytorch_model_summary import summary

# Archs not in Pytorch
from efficientnet_pytorch import EfficientNet

# External functions
from scripts.utils import *

In [2]:
print(torch.__version__)

import sys
print(sys.version)

1.5.1
3.8.5 (default, Jul 28 2020, 12:59:40) 
[GCC 9.3.0]


## Data paths and hyperparameters

#### Hyperparameters and dataset details. 

In [3]:
# Dataset details
dataset_version = 'original' # original or augmented
img_shape = (224,224)
img_size = str(img_shape[0])+"x"+str(img_shape[1])

# Root directory of dataset
data_dir = '/home/yibbtstll/venvs/pytorch_gpu/CySDeepBacterial/Dataset/DIBaS/'

train_batch_size = 32
val_test_batch_size = 32
feature_extract = False
pretrained = True
h_epochs = 10
kfolds = 10

## Data preparation and loading

#### Defining transforms and creating dataloaders

In [4]:
# Define transforms for input data
training_transforms = transforms.Compose([transforms.Resize((224,224), Image.LANCZOS),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], 
                                                               [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
total_set = datasets.ImageFolder(data_dir, transform=training_transforms)

# Defining folds
splits = KFold(n_splits = kfolds, shuffle = True, random_state = 42)

#### Visualizing the target classes in dataset

In [5]:
train_labels = {value : key for (key, value) in total_set.class_to_idx.items()}
    
print(len(train_labels)) 
print(train_labels)

32
{0: 'Acinetobacter.baumanii', 1: 'Actinomyces.israeli', 2: 'Bacteroides.fragilis', 3: 'Bifidobacterium.spp', 4: 'Clostridium.perfringens', 5: 'Enterococcus.faecalis', 6: 'Enterococcus.faecium', 7: 'Escherichia.coli', 8: 'Fusobacterium', 9: 'Lactobacillus.casei', 10: 'Lactobacillus.crispatus', 11: 'Lactobacillus.delbrueckii', 12: 'Lactobacillus.gasseri', 13: 'Lactobacillus.jehnsenii', 14: 'Lactobacillus.johnsonii', 15: 'Lactobacillus.paracasei', 16: 'Lactobacillus.plantarum', 17: 'Lactobacillus.reuteri', 18: 'Lactobacillus.rhamnosus', 19: 'Lactobacillus.salivarius', 20: 'Listeria.monocytogenes', 21: 'Micrococcus.spp', 22: 'Neisseria.gonorrhoeae', 23: 'Porfyromonas.gingivalis', 24: 'Propionibacterium.acnes', 25: 'Proteus', 26: 'Pseudomonas.aeruginosa', 27: 'Staphylococcus.aureus', 28: 'Staphylococcus.epidermidis', 29: 'Staphylococcus.saprophiticus', 30: 'Streptococcus.agalactiae', 31: 'Veionella'}


## Model definition and inicialization

#### Freezing pre-trained parameters, finetunning the classifier to output 32 classes.

In [6]:
# Freeze pretrained model parameters to avoid backpropogating through them
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        print("Setting grad to false.")
        for param in model.parameters():
            param.requires_grad = False
    
    return model

def get_device():
    # Model and criterion to GPU
    if torch.cuda.is_available():
        return 'cuda'
    else:
        return 'cpu'

In [7]:
def load_model():
    # Transfer Learning
    model = EfficientNet.from_pretrained('efficientnet-b0')
    
    # Mode
    model = set_parameter_requires_grad(model, feature_extract)
    
    # Fine tuning
    # Build custom classifier
    model._fc = nn.Linear(in_features=1280,
                        out_features=32)
    return model

def create_optimizer(model):
    # Parameters to update
    params_to_update = model.parameters()

    if feature_extract:
        params_to_update = []
        for param in model.parameters():
            if param.requires_grad == True:
                params_to_update.append(param)

    else:
        n_params = 0
        for param in model.parameters():
            if param.requires_grad == True:
                n_params += 1


    # Loss function and gradient descent

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(params_to_update, 
                          lr=0.001, 
                          weight_decay=0.000004)
    
    return criterion.to(get_device()), model.to(get_device()), optimizer

## Training, validation and test functions

In [8]:
# Variables to store fold scores
train_acc = []
test_top1_acc = []
test_top5_acc = []
test_precision = []
test_recall = []
test_f1 = []
times = []

for fold, (train_idx, valid_idx) in enumerate(splits.split(total_set)):
    
    start_time = time.time()
    
    print('Fold : {}'.format(fold))
    
    # Train and val samplers
    train_sampler = SubsetRandomSampler(train_idx)
    print("Samples in training:", len(train_sampler))
    valid_sampler = SubsetRandomSampler(valid_idx)
    print("Samples in test:", len(valid_sampler))
    
    # Train and val loaders
    train_loader = torch.utils.data.DataLoader(
                      total_set, batch_size=train_batch_size, sampler=train_sampler)
    valid_loader = torch.utils.data.DataLoader(
                      total_set, batch_size=1, sampler=valid_sampler)
    
    device = get_device()
    
    criterion, model, optimizer = create_optimizer(load_model())
    
    # Training
    for epoch in range(h_epochs):
        
        model.train()
        running_loss = 0.0
        running_corrects = 0
        trunning_corrects = 0
        
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
            running_loss += loss.item() * inputs.size(0)
            running_corrects += (preds == labels).sum()
            trunning_corrects += preds.size(0)
            

        epoch_loss = running_loss / trunning_corrects
        epoch_acc = (running_corrects.double()*100) / trunning_corrects
        train_acc.append(epoch_acc.item())
        
        print('\t\t Training: Epoch({}) - Loss: {:.4f}, Acc: {:.4f}'.format(epoch, epoch_loss, epoch_acc))
        
        # Validation
        
        model.eval()  
        
        vrunning_loss = 0.0
        vrunning_corrects = 0
        num_samples = 0
        
        for data, labels in valid_loader:
            
            data = data.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            with torch.no_grad():
                outputs = model(data)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
            vrunning_loss += loss.item() * data.size(0)
            vrunning_corrects += (preds == labels).sum()
            num_samples += preds.size(0)
            
        vepoch_loss = vrunning_loss/num_samples
        vepoch_acc = (vrunning_corrects.double() * 100)/num_samples
        
        print('\t\t Validation({}) - Loss: {:.4f}, Acc: {:.4f}'.format(epoch, vepoch_loss, vepoch_acc))
    
    # Calculating and appending scores to this fold
    scores = get_scores(model, valid_loader, total_set)
    
    test_top1_acc.append(scores[0])
    test_top5_acc.append(scores[1])
    test_precision.append(scores[2])
    test_recall.append(scores[3])
    test_f1.append(scores[4])
    
    time_fold = time.time() - start_time
    times.append(time_fold)
    print("Total time per fold: %s seconds." %(time_fold))

Fold : 0
Samples in training: 602
Samples in test: 67
Loaded pretrained weights for efficientnet-b0
		 Training: Epoch(0) - Loss: 2.0659, Acc: 54.6512
		 Validation(0) - Loss: 2.8162, Acc: 14.9254
		 Training: Epoch(1) - Loss: 0.5333, Acc: 87.0432
		 Validation(1) - Loss: 2.0726, Acc: 31.3433
		 Training: Epoch(2) - Loss: 0.3502, Acc: 91.3621
		 Validation(2) - Loss: 1.9500, Acc: 47.7612
		 Training: Epoch(3) - Loss: 0.2385, Acc: 93.6877
		 Validation(3) - Loss: 1.4259, Acc: 55.2239
		 Training: Epoch(4) - Loss: 0.2521, Acc: 91.6944
		 Validation(4) - Loss: 1.1600, Acc: 65.6716
		 Training: Epoch(5) - Loss: 0.1420, Acc: 96.6777
		 Validation(5) - Loss: 0.3932, Acc: 88.0597
		 Training: Epoch(6) - Loss: 0.0807, Acc: 97.8405
		 Validation(6) - Loss: 0.6703, Acc: 86.5672
		 Training: Epoch(7) - Loss: 0.0962, Acc: 97.1761
		 Validation(7) - Loss: 0.9338, Acc: 80.5970
		 Training: Epoch(8) - Loss: 0.0871, Acc: 97.1761
		 Validation(8) - Loss: 0.7035, Acc: 86.5672
		 Training: Epoch(9) - Los

		 Validation(2) - Loss: 2.3106, Acc: 44.7761
		 Training: Epoch(3) - Loss: 0.2069, Acc: 93.1894
		 Validation(3) - Loss: 3.2814, Acc: 38.8060
		 Training: Epoch(4) - Loss: 0.1821, Acc: 94.5183
		 Validation(4) - Loss: 1.2449, Acc: 67.1642
		 Training: Epoch(5) - Loss: 0.1606, Acc: 95.8472
		 Validation(5) - Loss: 1.3256, Acc: 74.6269
		 Training: Epoch(6) - Loss: 0.2046, Acc: 94.3522
		 Validation(6) - Loss: 0.5253, Acc: 83.5821
		 Training: Epoch(7) - Loss: 0.1061, Acc: 97.5083
		 Validation(7) - Loss: 0.9302, Acc: 76.1194
		 Training: Epoch(8) - Loss: 0.1079, Acc: 96.5116
		 Validation(8) - Loss: 1.7104, Acc: 62.6866
		 Training: Epoch(9) - Loss: 0.1388, Acc: 96.1794
		 Validation(9) - Loss: 0.3053, Acc: 91.0448
Finished.
Total time per fold: 270.3141601085663 seconds.
Fold : 8
Samples in training: 602
Samples in test: 67
Loaded pretrained weights for efficientnet-b0
		 Training: Epoch(0) - Loss: 2.0797, Acc: 51.3289
		 Validation(0) - Loss: 3.0111, Acc: 16.4179
		 Training: Epoch(1

In [9]:
print("Train accuracy average: ", np.mean(train_acc) / 100)
print("Top-1 test accuracy average: ", np.mean(test_top1_acc))
print("Top-5 test accuracy average: ", np.mean(test_top5_acc))
print("Weighted Precision test accuracy average: ", np.mean(test_precision))
print("Weighted Recall test accuracy average: ", np.mean(test_recall))
print("Weighted F1 test accuracy average: ", np.mean(test_f1))
print("Average time per fold (seconds):", np.mean(times))

Train accuracy average:  0.9050825606188326
Top-1 test accuracy average:  0.8668928086838534
Top-5 test accuracy average:  0.9940298507462687
Weighted Precision test accuracy average:  0.8837668798862829
Weighted Recall test accuracy average:  0.8668928086838534
Weighted F1 test accuracy average:  0.8558060264776683
Average time per fold (seconds): 268.0622427940369
